# A팀 (하드캐리팀)
# Otto group product classification
## 강호영 이상훈 유영재

In [0]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import math
import os
from random import randint

## 데이터 로드 및 특징

In [2]:
preTrain = pd.read_csv("./train_targetInt.csv")
preTrain.drop('Unnamed: 0', axis=1, inplace=True)
preTrain.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1


In [0]:
preTrain.describe()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
count,61878.000000,61878.00000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,...,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000
mean,30939.500000,0.38668,0.263066,0.901467,0.779081,0.071043,0.025696,0.193704,0.662433,1.011296,...,0.532306,1.128576,0.393549,0.874915,0.457772,0.812421,0.264941,0.380119,0.126135,4.842335
std,17862.784315,1.52533,1.252073,2.934818,2.788005,0.438902,0.215333,1.030102,2.255770,3.474822,...,1.900438,2.681554,1.575455,2.115466,1.527385,4.597804,2.045646,0.982385,1.201720,2.510794
min,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,15470.250000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,30939.500000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
75%,46408.750000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000
max,61878.000000,61.00000,51.000000,64.000000,70.000000,19.000000,10.000000,38.000000,76.000000,43.000000,...,55.000000,65.000000,67.000000,30.000000,61.000000,130.000000,52.000000,19.000000,87.000000,9.000000


In [15]:
preTest = pd.read_csv("./test.csv")
preTest.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
0,1,0,0,0,0,0,0,0,0,0,...,0,0,11,1,20,0,0,0,0,0
1,2,2,2,14,16,0,0,0,0,0,...,0,0,0,0,0,4,0,0,2,0
2,3,0,1,12,1,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,1
3,4,0,0,0,1,0,0,0,0,0,...,0,3,1,0,0,0,0,0,0,0
4,5,1,0,0,1,0,0,1,2,0,...,0,0,0,0,0,0,0,9,0,0


## 데이터 전처리 > log scailing

In [0]:
def log_scale(data):
  for feature in data.columns[1:94]:
    scaleDummy = []
    for n in data[feature]:
      scaleDummy.append(math.log10(n+1))
    data[feature] = scaleDummy
  return data

In [4]:
trainRaw = log_scale(preTrain)
trainRaw.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,0.30103,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.00000,0.0,...,0.30103,0.00000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,1
1,2,0.00000,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.30103,0.0,...,0.00000,0.00000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,1
2,3,0.00000,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.30103,0.0,...,0.00000,0.00000,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,1
3,4,0.30103,0.0,0.0,0.30103,0.845098,0.30103,0.778151,0.00000,0.0,...,0.00000,0.30103,0.477121,0.0,0.0,0.00000,0.0,0.0,0.0,1
4,5,0.00000,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.00000,0.0,...,0.30103,0.00000,0.000000,0.0,0.0,0.30103,0.0,0.0,0.0,1


In [0]:
trainRaw.describe()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
count,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,...,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000
mean,30939.500000,0.075185,0.049744,0.121108,0.117062,0.017380,0.006735,0.038128,0.120869,0.118816,...,0.096580,0.199564,0.080011,0.157608,0.093936,0.081122,0.036810,0.088084,0.024161,4.842335
std,17862.784315,0.188445,0.159492,0.281224,0.259088,0.083421,0.050803,0.138082,0.232394,0.292268,...,0.215920,0.283516,0.184629,0.265143,0.198915,0.250462,0.151920,0.181542,0.107565,2.510794
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,15470.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,30939.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
75%,46408.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.301030,0.000000,...,0.000000,0.301030,0.000000,0.301030,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000
max,61878.000000,1.792392,1.716003,1.812913,1.851258,1.301030,1.041393,1.591065,1.886491,1.643453,...,1.748188,1.819544,1.832509,1.491362,1.792392,2.117271,1.724276,1.301030,1.944483,9.000000


In [16]:
test = log_scale(preTest)
test.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
0,1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,...,0.0,0.00000,1.079181,0.30103,1.322219,0.00000,0.0,0.0,0.000000,0.00000
1,2,0.477121,0.477121,1.176091,1.230449,0.0,0.0,0.00000,0.000000,0.0,...,0.0,0.00000,0.000000,0.00000,0.000000,0.69897,0.0,0.0,0.477121,0.00000
2,3,0.000000,0.301030,1.113943,0.301030,0.0,0.0,0.00000,0.000000,0.0,...,0.0,0.00000,0.000000,0.00000,0.477121,0.00000,0.0,0.0,0.000000,0.30103
3,4,0.000000,0.000000,0.000000,0.301030,0.0,0.0,0.00000,0.000000,0.0,...,0.0,0.60206,0.301030,0.00000,0.000000,0.00000,0.0,0.0,0.000000,0.00000
4,5,0.301030,0.000000,0.000000,0.301030,0.0,0.0,0.30103,0.477121,0.0,...,0.0,0.00000,0.000000,0.00000,0.000000,0.00000,0.0,1.0,0.000000,0.00000


# Deep Learning 적용

## Relu 3 layers Adam 
## 256 Nodes Xavier

In [0]:
tf.set_random_seed(777)

# xavier 변수 초기화
tf.reset_default_graph()

global_step = tf.Variable(0, trainable=False, name='global_step')

nb_classes = 10

X = tf.placeholder(tf.float32, [None, 93])
Y = tf.placeholder(tf.int32, [None, 1])

Y_one_hot = tf.one_hot(Y, nb_classes)

Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

W1 = tf.get_variable("W1", shape=[93, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[256, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[256, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(L2, W3) + b3
hypothesis = tf.nn.softmax(logits)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(3001):
    sess.run(optimizer, feed_dict={X: tmpTrain.iloc[:, 1:94], Y: tmpTrain.iloc[:, -1:]})
    if step % 1000 == 0 or (step % 100 == 0 and step < 1000):
        loss, acc, result = sess.run([cost, accuracy, hypothesis], 
                                     feed_dict={X: tmpTrain.iloc[:, 1:94], 
                                                Y: tmpTrain.iloc[:, -1:]})
        print("Step: {:5}, Loss: {:.3f}, Acc: {:.2%}".format(step, loss, acc))

Step:     0, Loss: 2.140, Acc: 20.91%
Step:   100, Loss: 0.957, Acc: 68.87%
Step:   200, Loss: 0.694, Acc: 73.65%
Step:   300, Loss: 0.627, Acc: 76.00%
Step:   400, Loss: 0.589, Acc: 77.44%
Step:   500, Loss: 0.561, Acc: 78.44%
Step:   600, Loss: 0.536, Acc: 79.29%
Step:   700, Loss: 0.508, Acc: 80.29%
Step:   800, Loss: 0.480, Acc: 81.37%
Step:   900, Loss: 0.451, Acc: 82.56%
Step:  1000, Loss: 0.423, Acc: 83.84%
Step:  2000, Loss: 0.223, Acc: 92.50%
Step:  3000, Loss: 0.127, Acc: 96.04%


## Relu 3layers Adam Xavier
## 256Nodes Xavier Drop out

In [0]:
tf.set_random_seed(777)

# xavier 변수 초기화
tf.reset_default_graph()

global_step = tf.Variable(0, trainable=False, name='global_step')

# 입력 출력 데이터 분류

nb_classes = 10

X = tf.placeholder(tf.float32, [None, 93])
Y = tf.placeholder(tf.int32, [None, 1])
keep_prob = tf.placeholder(tf.float32)

Y_one_hot = tf.one_hot(Y, nb_classes)

Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

W1 = tf.get_variable("W1", shape=[93, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[256, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[256, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(L2, W3) + b3
hypothesis = tf.nn.softmax(logits)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

(61878, 93) (61878, 1)


In [0]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(10001):
    sess.run(optimizer, feed_dict={X: xData, Y: yData, keep_prob: 0.75})
    if step % 1000 == 0 or (step % 100 == 0 and step < 1000):
        loss, acc, result = sess.run([cost, accuracy, hypothesis], 
                                     feed_dict={X: xData, Y: yData, keep_prob: 0.75})
        print("Step: {:5}, Loss: {:.3f}, Acc: {:.2%}".format(
            step, loss, acc))

# 과적합 방지

## Validation data 생성

In [0]:
def validIndex(data, ratio):
  numList = range(len(data))
  numExtract = round(len(data)*ratio)
  numSample = random.sample(numList, numExtract)
  onlyTrain = []
  for n in numList:
    if n not in numSample: onlyTrain.append(n)
  return data.iloc[numSample], data.iloc[onlyTrain]

In [7]:
validation, remain = validIndex(trainRaw, 0.15)
len(validation), len(remain)

(9282, 52596)

## validation 데이터를 제외한 train 데이터 분할

In [9]:
tmpTrain, remain = validIndex(remain, 1.0)
len(tmpTrain), len(remain)

(52596, 0)

## weight 저장 폴더 생성

In [0]:
# 학습모델 저장폴더 생성
MODEL_DIR = './models/nodrop'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

## 학습된 weight를 validation에 적용 및 정확도 출력

In [74]:
accracy = sess.run(accuracy, feed_dict={X: validation.iloc[:, 1:94], 
                                        Y: validation.iloc[:, -1:]})
accracy

0.7955182

## 다양한 횟수로 학습한 것 중 최적의 weight를 출력하는 함수

In [0]:
sess = tf.Session()

In [0]:
# 여러 스텝 중 최적의 weight를 구하는 함수
def check_acc(num):
  accList = []
  for n in range(1, num+1):
    realStep = 10*n+1
    chkpoint = tf.train.get_checkpoint_state('./models/nodrop')
    if chkpoint and tf.train.checkpoint_exists(chkpoint.model_checkpoint_path):
      saver.restore(sess, chkpoint.model_checkpoint_path)
    else:
      sess.run(tf.global_variables_initializer())
    
    for step in range(realStep):
      sess.run(optimizer, feed_dict={X: tmpTrain.iloc[:, 1:94], Y: tmpTrain.iloc[:, -1:]})
          
    accracy = sess.run(accuracy, feed_dict={X: validation.iloc[:, 1:94], 
                                            Y: validation.iloc[:, -1:]})
    accList.append(accracy)
    
    if n % 10 ==0: print("{} {}".format(n, len(accList)), end ='\t')
  return accList

In [69]:
accList = check_acc(30)

INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
10 10	INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
INFO:tensorflow:Res

In [70]:
accList

[0.44343892,
 0.544387,
 0.6141995,
 0.68024135,
 0.71805644,
 0.7336781,
 0.739819,
 0.74962294,
 0.75404006,
 0.76276666,
 0.7656755,
 0.7692308,
 0.77343243,
 0.7763413,
 0.78075844,
 0.78291315,
 0.7853911,
 0.7868994,
 0.7885154,
 0.7906701,
 0.7900237,
 0.79077786,
 0.79228616,
 0.7918552,
 0.7927171,
 0.7942254,
 0.7955182,
 0.79530275,
 0.79541045,
 0.79530275]

In [71]:
accList.index(max(accList)) +1

27

## weight 보존

In [72]:
# 최적의 weight에 다가가기 위한 함수
# 파일 체크
chkpoint = tf.train.get_checkpoint_state('./models/nodrop')
if chkpoint and tf.train.checkpoint_exists(chkpoint.model_checkpoint_path):
    saver.restore(sess, chkpoint.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
# sess.run(tf.global_variables_initializer())

for step in range(271):
    sess.run(optimizer, feed_dict={X: tmpTrain.iloc[:, 1:94], Y: tmpTrain.iloc[:, -1:]})
    if step % 1000 == 0 or (step % 100 == 0 and step < 1000):
        loss, acc, result = sess.run([cost, accuracy, hypothesis], 
                                     feed_dict={X: tmpTrain.iloc[:, 1:94], 
                                                Y: tmpTrain.iloc[:, -1:]})
        print("Step: {:5}, Loss: {:.3f}, Acc: {:.2%}".format(step, loss, acc))

INFO:tensorflow:Restoring parameters from ./models/nodrop/lay3.1e3.step1-0
Step:     0, Loss: 2.091, Acc: 26.25%
Step:   100, Loss: 0.649, Acc: 75.60%
Step:   200, Loss: 0.544, Acc: 79.23%


In [73]:
accracy = sess.run(accuracy, feed_dict={X: validation.iloc[:, 1:94], 
                                        Y: validation.iloc[:, -1:]})
accracy

0.7955182

In [62]:
# weight 보존해서 학습
saver = tf.train.Saver(tf.global_variables())
saver.save(sess, './models/nodrop/lay3.1e3.step1', global_step=global_step)

'./models/nodrop/lay3.1e3.step1-0'

## train label을 이용해 대략적인 test

In [0]:
justX = sess.run(prediction, feed_dict={X: xData})
justX = pd.Series(justX)
justX.value_counts()

2    16402
6    14131
8     8450
3     7839
9     4952
7     2821
5     2740
4     2624
1     1919
dtype: int64

In [0]:
justY = sess.run(tf.argmax(Y_one_hot, 1), feed_dict={Y: yData})
justY = pd.Series(justY)
justY.value_counts()

2    16122
6    14135
8     8464
3     8004
9     4955
7     2839
5     2739
4     2691
1     1929
dtype: int64

# 제출물 생성

In [50]:
xTest = test.iloc[:, 1:94].astype(float)
len(xTest)

144368

In [0]:
# No drop out
result = sess.run(hypothesis, feed_dict= {X: xTest})

In [0]:
# drop out
result = sess.run(hypothesis, feed_dict= {X: xTest, keep_prob: 1.0})

In [76]:
result[0]

array([9.7809592e-09, 8.7840839e-05, 2.4050014e-01, 1.8005252e-01,
       5.6795180e-01, 2.8885220e-06, 3.7401615e-04, 1.0959112e-02,
       7.0434035e-05, 1.2803835e-06], dtype=float32)

## 제출 포맷 생성 코드

In [77]:
# 출력 데이터 형태 만들기
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []
c7 = []
c8 = []
c9 = []
iden = []

for n in range(len(result)):
  iden.append(n+1)
  c1.append(result[n][1])
  c2.append(result[n][2])
  c3.append(result[n][3])
  c4.append(result[n][4])
  c5.append(result[n][5])
  c6.append(result[n][6])
  c7.append(result[n][7])
  c8.append(result[n][8])
  c9.append(result[n][9])
  
resDf = pd.DataFrame({'id' : iden,
                      'Class_1' : c1,
                      'Class_2' : c2,
                      'Class_3' : c3,
                      'Class_4' : c4,
                      'Class_5' : c5,
                      'Class_6' : c6,
                      'Class_7' : c7,
                      'Class_8' : c8,
                      'Class_9' : c9})

cols = [resDf.columns[-1]] + [col for col in resDf if col != resDf.columns[-1]]
resDf = resDf[cols]
resDf.set_index('id', inplace=True)
resDf.head()

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
id,,,,,,,,,
1,0.000088,2.405001e-01,1.800525e-01,5.679518e-01,2.888522e-06,0.000374,0.010959,0.000070,0.000001
2,0.001573,2.083430e-03,7.249854e-05,2.135389e-04,5.539732e-04,0.349449,0.000586,0.623886,0.021583
3,0.000005,3.645461e-08,4.149065e-08,2.535489e-08,2.941408e-09,0.999948,0.000003,0.000031,0.000013
4,0.000069,5.099242e-01,4.788100e-01,1.007003e-02,2.786970e-06,0.000014,0.000126,0.000009,0.000975
5,0.215655,1.866454e-04,9.192061e-05,2.747643e-05,3.187786e-07,0.004133,0.000989,0.021184,0.757733


## 제출물 csv 파일 출력

In [0]:
resDf.to_csv('./sample_data/relu256_nodrop3_vali271.csv', sep=',', encoding='utf-8-sig')

## 제출물 파일 체크

In [0]:
check = pd.read_csv('sample_data/relu256_nodrop3_vali1300.csv')
check.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,7.209775e-09,1.121509e-03,1.055122e-01,8.933249e-01,7.789883e-14,2.306713e-09,4.141774e-05,1.256526e-08,2.237493e-12
1,2,1.157383e-10,1.289152e-02,2.718332e-05,8.403281e-09,5.520099e-08,2.744112e-08,1.313812e-08,9.842026e-01,2.878617e-03
2,3,2.669149e-12,1.285589e-15,5.046788e-14,2.350110e-14,2.054441e-17,1.000000e+00,2.663531e-11,1.103965e-09,6.781395e-11
3,4,6.177626e-12,6.341051e-03,9.936214e-01,1.049839e-05,8.528034e-14,1.061285e-16,5.544301e-11,1.746582e-08,2.695237e-05
4,5,1.823590e-02,1.540355e-10,3.197002e-09,2.086919e-12,3.500919e-22,2.851185e-09,3.509965e-07,8.099020e-01,1.718618e-01
